## Querying a knowledge graph using SPARQL

As seen in Tutorial 3a with rdflib, SPARQL can be used to access information contained in a graph. It is the standard query language for knowledge graphs.

If not done alreday, please select the SPARQL kernel now (top right) to perform this Tutorial. The installation instructions for the SPARQL kernel can be found in the github repo (`https://github.com/michaelakuempel/ActionableKnowledgeRepresentation`)

Like in the previous Tutorial, we start with loading the Knowledge Graph and querying for all subclasses of the main "Thing" class:

In [1]:
# set parameters for kernel to display results
%endpoint https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
%format default
%display table 
%show 10

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT * WHERE{
    ?x rdfs:subClassOf owl:Thing.
}

Endpoint set to: https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
Request format: DEFAULT
Display: table
Result maximum size: 10

x
http://purl.org/ProductKG/RecipeOn#IngredientSet
http://purl.org/ProductKG/allergen#Allergen
http://purl.org/ProductKG/allergen#allergy
http://purl.org/ProductKG/food-nutrition#Food
http://purl.org/ProductKG/location#retail_lab_IAI
http://purl.org/ProductKG/nonfoodingredient#Ingredient
http://purl.org/ProductKG/nutrition#nutritional_component
http://purl.org/ProductKG/packaging.owl#Packaging
http://purl.org/ProductKG/product-taxonomy#Product
http://purl.org/ProductKG/product-taxonomy.owl#Product


We proceed to again query all subclasses of `http://purl.org/goodrelations/v1#ProductOrService` and all subclasses of the retrieved classes

In [2]:
%endpoint https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
%format default
%display table 
%show 10

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX gr: <http://purl.org/goodrelations/v1#>

SELECT * WHERE{
    ?cls rdfs:subClassOf gr:ProductOrService.
    ?subcls rdfs:subClassOf ?cls.
}

Endpoint set to: https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
Request format: DEFAULT
Display: table
Result maximum size: 10

cls,subcls
http://knowrob.org/kb/shop.owl#CareProduct,http://purl.org/ProductKG/product-taxonomy#body_care
http://purl.org/ProductKG/product-taxonomy#Household,http://purl.org/ProductKG/product-taxonomy#paper_products
http://purl.org/ProductKG/product-taxonomy#Household,http://knowrob.org/kb/shop.owl#Candle
http://purl.org/ProductKG/product-taxonomy#Household,http://knowrob.org/kb/shop.owl#Filter
http://purl.org/ProductKG/product-taxonomy#Household,http://knowrob.org/kb/shop.owl#HouseholdFoil
http://purl.org/ProductKG/product-taxonomy#Household,http://knowrob.org/kb/shop.owl#Lightbulb
http://purl.org/ProductKG/product-taxonomy#Household,http://purl.org/ProductKG/product-taxonomy#bags
http://purl.org/ProductKG/product-taxonomy#Household,http://purl.org/ProductKG/product-taxonomy#batteries
http://purl.org/ProductKG/product-taxonomy#baby/child,http://knowrob.org/kb/shop.owl#BabyAccessory
http://purl.org/ProductKG/product-taxonomy#baby/child,http://purl.org/ProductKG/product-taxonomy#baby_care


#### Application example

Let's consider the example of being in the store and looking at a product we are interested in. Interesting properties to query might be label of the product.

- Search for the product with a given EAN (barcode) of "4010355090935"
- products can be found in `http://purl.org/ProductKG/product-taxonomy#`
- names for products use the good relations vocabulary (gr:name)
- labels are found in `http://purl.org/ProductKG/label#`

In [3]:
%endpoint https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
%format default
%display table 
%show 10

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX gr: <http://purl.org/goodrelations/v1#>
PREFIX tax: <http://purl.org/ProductKG/product-taxonomy#>
PREFIX lbl: <http://purl.org/ProductKG/label#>

SELECT ?instance ?name ?label WHERE{
    ?instance gr:hasEAN_UCC-13 "4010355090935".
    ?instance gr:name ?name.
    ?instance lbl:has_label ?label.
}

Endpoint set to: https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
Request format: DEFAULT
Display: table
Result maximum size: 10

instance,name,label
http://purl.org/ProductKG/product-taxonomy#P4010355090935,Balea Teint Perfektion Wimpernserum,http://purl.org/ProductKG/label#FSC_Mix
http://purl.org/ProductKG/product-taxonomy#P4010355090935,Balea Teint Perfektion Wimpernserum,http://purl.org/ProductKG/label#vegetarian


For the same product, we can search for contained ingredients and allergens:

- Search for the product with a given EAN (barcode) of "4010355090935"
- products can be found in `http://purl.org/ProductKG/product-taxonomy#`
- ingredients are found in `http://purl.org/ProductKG/nonfoodingredient#`
- allergens are linked in `http://purl.org/ProductKG/allergen#`

In [4]:
%endpoint https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
%format default
%display table 
%show 10

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX gr: <http://purl.org/goodrelations/v1#>
PREFIX tax: <http://purl.org/ProductKG/product-taxonomy#>
PREFIX in: <http://purl.org/ProductKG/nonfoodingredient#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?ingredient ?iclass ?allergen ?aclass WHERE{
    ?instance gr:hasEAN_UCC-13 "4010355090935".
    ?instance in:has_ingredient ?ingredient.
    OPTIONAL{?ingredient rdf:type ?iclass.
             FILTER(?iclass != owl:NamedIndividual && ?iclass != in:Ingredient) }
    OPTIONAL{?allergen owl:sameAs ?ingredient.
             ?allergen rdf:type ?aclass.
             FILTER(?aclass != owl:NamedIndividual)  }
}

Endpoint set to: https://api.krr.triply.cc/datasets/mkumpel/ProductKG/sparql
Request format: DEFAULT
Display: table
Result maximum size: 10

ingredient,iclass,allergen,aclass
http://purl.org/ProductKG/nonfoodingredient#benzyl_alcohol,http://purl.org/ProductKG/nonfoodingredient#Alcohol,http://purl.org/ProductKG/allergen#benzyl_alcohol,http://purl.org/ProductKG/nonfoodingredient#Benzyl_alcohol
http://purl.org/ProductKG/nonfoodingredient#benzyl_alcohol,http://purl.org/ProductKG/nonfoodingredient#Benzyl_alcohol,http://purl.org/ProductKG/allergen#benzyl_alcohol,http://purl.org/ProductKG/nonfoodingredient#Benzyl_alcohol
http://purl.org/ProductKG/nonfoodingredient#acrylates,http://purl.org/ProductKG/nonfoodingredient#Polymer,,
http://purl.org/ProductKG/nonfoodingredient#alkyl_acrylate_crosspolymer,http://purl.org/ProductKG/nonfoodingredient#Polymer,,
http://purl.org/ProductKG/nonfoodingredient#benzoic_acid,http://purl.org/ProductKG/nonfoodingredient#Acid,,
http://purl.org/ProductKG/nonfoodingredient#benzoic_acid,http://purl.org/ProductKG/nonfoodingredient#Benzoic_acid,,
http://purl.org/ProductKG/nonfoodingredient#c,http://purl.org/ProductKG/nonfoodingredient#Dye,,
http://purl.org/ProductKG/nonfoodingredient#dehydroacetic_acid,http://purl.org/ProductKG/nonfoodingredient#Acid,,
http://purl.org/ProductKG/nonfoodingredient#allantoin,,,
http://purl.org/ProductKG/nonfoodingredient#aqua,,,


## Exercise

Get information about price and stock for the same product ("4010355090935").
- Use the prefix `location` to access inventory data from the IRI `http://purl.org/ProductKG/location#`
- Use the `http://purl.org/ProductKG/location#has_price` property to get the price of a product
- Use the `http://purl.org/ProductKG/location#has_stock` property to get the stock of a product
